# Convolutional Neural Networks

## Load Data

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import KFold

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


2024-07-28 21:04:16.074077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 21:04:16.087331: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 21:04:16.087350: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 21:04:16.096854: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-28 21:04:16.758427: W tensorflow/compiler/tf

Num GPUs Available:  1


2024-07-28 21:04:17.633457: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 21:04:17.667029: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 21:04:17.669876: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
images_train = np.expand_dims(np.load("../data/images/images_train.npy") / 255.0, -1)
images_test = np.expand_dims(np.load("../data/images/images_test.npy") / 255.0, -1)
num_samples_train = images_train.shape[0]
num_samples_test = images_test.shape[0]

y_train = np.load("../data/images/label_ids_train.npy")
y_test = np.load("../data/images/label_ids_test.npy")

train_indices = np.random.permutation(num_samples_train)
images_train = images_train[train_indices]
y_train = y_train[train_indices]

test_indices = np.random.permutation(num_samples_test)
images_test = images_test[test_indices]
y_test = y_test[test_indices]



In [ ]:
labels_train = np.load("../data/images/labels_train.npy")
class_lookup = list(set(zip(y_train, labels_train)))
class_lookup = {id_value: class_value for id_value, class_value in class_lookup}
class_lookup


{4: 'dislocation',
 2: 'greenstick',
 8: 'impacted',
 9: 'avulsion',
 1: 'spiral',
 6: 'longitudinal',
 7: 'oblique',
 3: 'comminuted',
 5: 'pathological',
 0: 'hairline'}

In [ ]:
image_input = layers.Input(shape=(images_train.shape[1], images_train.shape[2], images_train.shape[3]))
x = layers.Conv2D(60, (3, 3), activation='relu')(image_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(120, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)

x = layers.Dense(40, activation='relu')(x)
output = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=[image_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit data to model
history = model.fit(images_train[train], y_train[train],
            batch_size=32,
            epochs=5,
            verbose=1)


In [ ]:
y_pred = model.predict([images_test])
y_pred = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report).transpose()
report.index = report.index.map(lambda x: class_lookup[int(x)] if x.isdigit() else x)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
report


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 285ms/step
Accuracy: 0.17857142857142858
Classification Report:


,precision,recall,f1-score,support
hairline,0.000000,0.000000,0.000000,10.000000
spiral,0.200000,0.166667,0.181818,12.000000
greenstick,0.277778,0.312500,0.294118,16.000000
comminuted,0.166667,0.285714,0.210526,14.000000
dislocation,0.272727,0.315789,0.292683,19.000000
pathological,0.176471,0.166667,0.171429,18.000000
longitudinal,0.000000,0.000000,0.000000,12.000000
oblique,0.100000,0.062500,0.076923,16.000000
impacted,0.166667,0.111111,0.133333,9.000000
avulsion,0.214286,0.214286,0.214286,14.000000
